Lab 6 Lab 7 Lab 8



In [8]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras import models, layers, regularizers, Model
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# ===== MOUNT GOOGLE DRIVE (if using Colab) =====
from google.colab import drive
drive.mount('/content/drive')

# ===== DATASET PATH =====
dataset_path = "/content/drive/MyDrive/NNDL Dataset/Dataset"

# ===== CHECK IF DATASET PATH EXISTS =====
if not os.path.exists(dataset_path):
    raise FileNotFoundError(f"Dataset path '{dataset_path}' does not exist! Check your Google Drive.")

# ===== CATEGORIES (Folder Names) =====
categories = [
    "gond painting", "kalighat painting", "kangra painting", "kerala mural",
    "madhubani painting", "mandana art drawing", "pichwai painting", "warli painting"
]

# ===== IMAGE LOADING SETTINGS =====
img_size = (128, 128)  # Resize images
valid_extensions = (".jpg", ".jpeg", ".png")  # Allowed file types

def load_images(folder_path, img_size):
    images, labels = [], []
    for category in categories:
        category_path = os.path.join(folder_path, category)
        if not os.path.exists(category_path):
            print(f"⚠️ Warning: {category_path} does not exist. Skipping.")
            continue
        for filename in os.listdir(category_path):
            if filename.lower().endswith(valid_extensions):
                img_path = os.path.join(category_path, filename)
                img = load_img(img_path, target_size=img_size)
                img_array = img_to_array(img) / 255.0  # Normalize
                images.append(img_array)
                labels.append(categories.index(category))
    return np.array(images), np.array(labels)

# ===== LOAD DATASET =====
X, y = load_images(dataset_path, img_size)
if len(X) == 0:
    raise ValueError("No images were loaded. Please check your dataset folder structure.")
print(f"✅ Loaded {len(X)} images with shape: {X.shape}")

# ===== SPLIT DATASET =====
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# ===== DEFINE CNN MODEL WITH REGULARIZATION =====
def create_cnn(input_shape):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001))(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(0.25)(x)

    x = layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(0.25)(x)

    x = layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(0.25)(x)

    x = layers.Flatten()(x)
    feature_output = layers.Dense(128, activation='relu', name='feature_layer')(x)
    x = layers.Dropout(0.5)(feature_output)
    outputs = layers.Dense(len(categories), activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model

# ===== BUILD & COMPILE CNN MODEL =====
cnn_model = create_cnn((128, 128, 3))
cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# ===== TRAIN CNN MODEL =====
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
cnn_model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), batch_size=32, callbacks=[early_stopping])

# ===== EXTRACT FEATURES FROM CNN MODEL =====
feature_extractor = Model(inputs=cnn_model.input, outputs=cnn_model.get_layer('feature_layer').output)
X_train_features = feature_extractor.predict(X_train)
X_val_features = feature_extractor.predict(X_val)

# ===== CLASSIFIERS =====
classifiers = {
    "kNN": KNeighborsClassifier(n_neighbors=5),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "SVM": SVC(kernel='linear'),
    "Naïve Bayes": GaussianNB()
}

# ===== TRAIN & EVALUATE CLASSIFIERS =====
for name, clf in classifiers.items():
    clf.fit(X_train_features, y_train)
    y_pred = clf.predict(X_val_features)

    accuracy = accuracy_score(y_val, y_pred)
    precision = precision_score(y_val, y_pred, average='weighted')
    recall = recall_score(y_val, y_pred, average='weighted')
    f1 = f1_score(y_val, y_pred, average='weighted')

    print(f"\n🔹 {name} Classifier Results:")
    print(f"   ✅ Accuracy: {accuracy:.4f}")
    print(f"   ✅ Precision: {precision:.4f}")
    print(f"   ✅ Recall: {recall:.4f}")
    print(f"   ✅ F1 Score: {f1:.4f}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


✅ Loaded 924 images with shape: (924, 128, 128, 3)
Epoch 1/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 44s 2s/step - accuracy: 0.1986 - loss: 6.8920 - val_accuracy: 0.1405 - val_loss: 6.9687
Epoch 2/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.2565 - loss: 2.2609 - val_accuracy: 0.0973 - val_loss: 16.9105
Epoch 3/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 81s 2s/step - accuracy: 0.2504 - loss: 2.2590 - val_accuracy: 0.0973 - val_loss: 21.8162
Epoch 4/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.3172 - loss: 2.0219 - val_accuracy: 0.0973 - val_loss: 26.3028
24/24 ━━━━━━━━━━━━━━━━━━━━ 7s 275ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 476ms/step

🔹 kNN Classifier Results:
   ✅ Accuracy: 0.3622
   ✅ Precision: 0.3732
   ✅ Recall: 0.3622
   ✅ F1 Score: 0.3446

🔹 Decision Tree Classifier Results:
   ✅ Accuracy: 0.2919
   ✅ Precision: 0.2849
   ✅ Recall: 0.2919
   ✅ F1 Score: 0.2862

🔹 Random Forest Classifier Results:
   ✅ Accuracy: 0.4270
   ✅ Precision: 0.4165
   ✅ Recall: 0.4270
   ✅ F1 Score: 0.3980


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
